# Flow network

1. Explanation
2. Drawing
3. Complexity
4. Network creation pseudocode
5. Network function pseudocode

[exercices](https://docs.google.com/document/d/1NKvKERKkDekM-mFw5rC3GuKwmRmRBtNHKSiDBauYnuw/edit)

## Examples (buch)

### Esquema Ford-Fulkerson

```python
def flow(graph, s, t):
    init flow through every edge to 0
    while there is an augmenting path in the residual network:
        increase flow acording to that path
```

### Ejemplo codigo Ford-Fulkerson

```python
def flow(graph, s, t):
    flow = {}
    for v in graph:
        for w in graph.adj(v):
            flow[(v, w)] = 0
    
    residual = copy(graph)
    
    while (path = obtener_camino(residual, s, t)) is not None:
        residual_flow = min_weight_of_all_edges(graph, path)
        for i in range(1, len(path)):
            if graph.has_edge(path[i-1], path[i]):
                flow[(path[i-1], path[i])] += residual_flow
                update_residual_graph(residual, path[i-1], path[i], residual_flow)
            else:
                flow[(path[i], path[i-1])] -= residual_flow
                update_residual_graph(residual, path[i], path[i-1], residual_flow)
    
    return flow


def update_residual_graph(graph, u, v, flow):
    last_weight = graph.weight_of_edge(u, v)
    
    if last_weight == flow:
        graph.remove_edge(u, v)
    else:
        graph.update_edge(u, v, last_weight - flow)
    
    if not graph.has_edge(v, u):
        graph.add_edge(v, u, flow)
    else:
        graph.update_edge(v, u, graph.weight_of_edge(v, u) + flow)
```

##### Complejidad:

La complejidad depende de como buscamos camino, si se usa el algoritmo de **Edmonds-Karp** osea el que usa BFS la complejidad es **O(V*E^2)**

###### O(E*V) si es bipartito

Tambien puede pasar que demore muchisimo o nunca converger **O(∞)** xd.

### Creacion de grafo residual

* Hay mas de una fuente -> superfuente
* Hay mas de un sumidero -> supersumidero
* Hay bucles (arista v, v) -> las saco
* Hay ciclos de 2 nodos (aristas v, w y w, v) -> creo un nodo intermediario imaginario (se forma un triangulo)

### Bipartite Matching

Se hace direccionado de una componente a otra componente, se agrega fuente y sumidero, todas las aristas en 1 y dsps se aplica ford-fulkerson y donezo

![EXAMPLE_5_MATCHING](img/EXAMPLE_5_MATCHING.png)

### Disjoint paths

* En dirigido se pone fuente y sumidero con aristas entre fuente y nodos con grado de entrada 0 (sin contar las aristas desde la fuente) el peso va a ser infinito, y el resto de los pesos sera 1, eso garantiza no reutilizar aristas.
* En no dirigido se hace lo mismo con la salvedad de que cuando se hace el BFS encontrando camino se tiene que tener en cuenta que siempre el nivel tiene que ir en aumento, porque si encontramos camino en el que en un momento el nivel disminuye significaria que estamos usando una arista para volver para atras (y se usaria 2 veces)

![EXAMPLE_5_DISJOINT](img/EXAMPLE_5_DISJOINT.png)

### Airline scheduling

Suponer que queremos schedulear cómo los aviones van de un aeropuerto a otro para cumplir horarios y demás. 
Podemos decir que podemos usar un avión para un segmento/vuelo i y luego para otro j si: 

* El destino de i y el origen de j son el mismo. 
* Podemos agregar un vuelo desde el destino de i al origen de j con tiempo suficiente. 

Decimos que el vuelo j es alcanzable desde el vuelo i si es posible usar el avión del vuelo i y después para el vuelo j. 

Problema: ¿Podemos cumplir con los m vuelos usando a lo sumo k aviones? -> [VID BUCH](https://www.youtube.com/watch?v=zHwYCJdSbRQ&ab_channel=AlgoritmosFiubaCursoBuchwald)

**SOLUCION**

1. Nuestras unidades de flujo son literalmente los aviones.
2. Ponemos las aristas de los vuelos que queremos si o si cumplir con cota mínima y capacidad = 1 (para forzar que se usen). 
3. Si otro vuelo es alcanzable por las reglas anteriores, ponemos otra arista de capacidad 1. 
4. Ponemos una fuente con aristas de capacidad 1 a los orígenes. 
5. Ponemos un sumidero con aristas de capacidad 1 desde los destinos.
6. La fuente tiene demanda -K y el sumidero K (es decir habra una superfuente dando flujo K a nuestra fuente, esto pq sino puede llegar a usar mas de los aviones que tiene la empresa).

![EXAMPLE_5_AIRLINE](img/EXAMPLE_5_AIRLINE.png)



## Ejs

### 1-

La sala de guardia de un hospital tiene que tener al menos un médico en todos los feriados y en los fines de semana largos de feriados. Cada profesional indica sus posibilidades: por ejemplo alguien puede estar de guardia en cualquier momento del fin de semana largo del 9 de julio (p. ej. disponibilidad de A para el 9 de julio = (Jueves 9/7, Viernes 10/7, Sábado 11/7, Domingo 12/7)), también puede suceder que alguien pueda sólo en parte (por ejemplo, disponibilidad de B para 9 de julio = (Jueves 9/7, Sábado 11/7, Domingo 12/7)). Aunque los profesionales tengan múltiples posibilidades, a cada uno se lo puede convocar para un solo día (se puede disponer de B sólo en uno de los tres días que indicó). Para ayudar a la sala de guardia a planificar cómo se cubren los feriados durante todo el año debemos resolver el problema de las guardias: Existen k períodos de feriados (por ejemplo, 9 de julio es un período de jueves 9/7 a domingo 12/7, en 2019 Día del Trabajador fue un período de 1 día: miércoles 1 de mayo, etc.). Dj es el conjunto de fechas que se incluyen en el período de feriado j-ésimo. Todos los días feriados son los que resultan de la unión de todos los Dj. Hay n médicos y cada médico i tiene asociado un conjunto Si de días feriados en los que puede trabajar (por ejemplo B tiene asociado los días Jueves 9/7, Sábado 11/7, Domingo 12/7, entre otros).

Proponer un algoritmo polinomial (usando flujo en redes) que toma esta información y devuelve qué profesional se asigna a cada día feriado (o informa que no es posible resolver el problema) sujeto a las restricciones:

* Ningún profesional trabajará más de F días feriados (F es un dato), y sólo en días en los que haya informado su disponibilidad.
* A ningún profesional se le asignará más de un feriado dentro de cada período Dj.


#### Solution

* Con ayuda de [Flor](https://github.com/fnpratto) jeje

##### Algoritmo:

Se crea la red de flujo donde los doctores estaran conectados a un superfuente que les dara flujo infinito ya que estos pueden trabajar en mas de un feriado, pero como no pueden trabajar 2 veces en el mismo feriado estos estaran conectados a unos nodos intermediarios dependiendo de que feriados pueden trabajar, y cada uno de estos nodos estaran conectados los dias de ese feriado que el doctor puede trabajar, la arista entre doctor feriado tiene capacidad 1 porque el doctor no puede trabajar mas de una vez en el mismo y las aristas entre nodo feriado a dia de ese feriado tambien tendran capacidad 1 ya que si un dia ya esta cubierto no hace falta que otro doctor lo cubra, luejo todos esos dias estaran conectados a un supersumidero con aristas de capacidad 1 por la misma razon anterior.

Una vez se tiene esta red de flujo se le calcula el flujo maximo usando un algoritmo como el de ford-fulkerson (o la variante edmond-karp) y si este flujo coinside con el numero de dias de feriados, significa que se logro cubrir todos, de lo contrario hay algun dia que no pudo ser cubierto.

##### Ejemplo:

![5_1](img/5_1.png)

##### Complejidad:

siendo V los nodos de la red de flujo y E las aristas de la misma:

La complejidad temporal es **O(V*E^2)** por edmond-karp

La complejidad espacial es **O(V+E)** por la creacion del grafo residual que copia el grafo de red de flujo.

##### Pseudocodigo:

```python
def cubre_los_dias(red, s, t):
    init flujo de cada arista a 0
    crear grafo residual a partir de la red

    mientras que haya un camino p entre s y t: # p el camino mas corto
        flujo residual = el minimo peso de todas las aristas del camino p
        si la red tiene la arista:
            flujo de la arista += flujo residual
        sino: # se usa arista antiparalela
            flujo de la arista -= flujo residual
        actualizar grafo residual
        
    devolver flujo == cantidad de feriados
```

### 2-

Definimos el Problema de la Evacuación de la siguiente manera: Se tiene un grafo dirigido G = (V, E) que describe una red de caminos. Tenemos una colección de nodos X ⊂ V que son los nodos poblados (ciudades) y otra colección de nodos S ⊂ V que son los nodos de refugio (supondremos que X y S son disjuntos). En caso de una emergencia queremos poder definir un conjunto de rutas de evacuación de los nodos poblados a los refugios. Un conjunto de rutas de evacuación es un conjunto de caminos en G tales que 

* cada nodo en X es el origen de un camino
* el último nodo de cada camino es un refugio (está en S)
* los caminos no comparten aristas entre sí. Se pide: dados G, X y S, mostrar cómo se puede decidir en tiempo polinomial si es posible construir un conjunto de rutas de evacuación (usar flujo en redes para eso. Construir la red adecuada).

#### Solution

* Con ayuda de [Flor](https://github.com/fnpratto) jeje

##### Algoritmo:

Se crea una red de flujo donde hay una superfuente que da flujo a todas las ciudades en X con capacidad 1 ya que cada ciudad solamente deberia usar un camino y no mas, cada ciudad tiene su camino hacia un refugio ya sea que ese camino pase o no por otras ciudades, todas las capacidades intermedias entre aristas de ciudad a ciudad o de ciudad a refugio tendran capacidad 1, luego todos los refugios seran unidos al supersumidero con capacidad infinita ya que a cada refugio le llegara un numero desconocido de ciudades. 

Se busca el flujo maximo atravez de la red con un algoritmo como ford-fulkerson (usando la variande de edmond-karp) y se ve si el flujo obtenido es igual a la cantidad de ciudades X, si lo es entonces hay camino para evacuar todas las ciudades, de lo contrario significa que hay ciudades que no pueden ser evacuadas sin exceder la capacidad de las rutas.

##### Ejemplo:

![5_2](img/5_2.png)

##### Complejidad:

La complejidad temporal sera **O(V*E^2)** por edmond-karp

La complejidad espacial sera **O(V+E)** porque se crea una copia del grafo dado para crear el grafo residual.

##### Pseudocodigo:

```python
def se_puede_evacuar(rutas, s, t):
    init flujo para cada arista en 0
    crear grafo residual

    mientras que haya camino de menor longitud entre s y t:
        flujo residual = min(todas las aristas del camino)
        para cada arista en el camino:
            si el grafo rutas tiene la arista:
                flujo de la arista += flujo residual
            sino:
                flujo de la arista -= flujo residual
            actualizar grafo residual

    devolver flujo == X
```

### 3-

Para un evento solidario un conjunto de n personas se ofrecieron a colaborar. En el evento hay m tareas a desarrollar. Cada tarea tiene un cupo máximo de personas que la puede realizar. A su vez cada persona tiene ciertas habilidades que la hacen adecuadas para un subconjunto de tareas. Proponga una solución mediante red de flujos que maximice la cantidad de personas asignadas a las tareas. ¿Hay forma de lograr asegurarnos un piso mínimo de personas en cada tarea? ¿Cómo impacta en la solución presentada en el punto anterior?

#### Solution

##### Algoritmo:

Se crea la red de flujo como un grafo dirijido donde habra multiples fuentes que representaran a las n personas por lo que se las une con una superfuente que les dara flujo infinito ya que una misma persona podra ser tenida en cuenta para mas de 1 tarea, las m tareas seran los multiples sumideros que tendran que ser conectados a un supersumidero con la capacidad de personas que pueden hacer esa tarea. Luego cada persona tendra una arista hacia las tareas para las que es adecuada y el peso de la misma sera 1, ya que el flujo en este caso representara las personas que hacen tareas, asi que 1 persona da 1 de flujo a una tarea especifica (o mas de una).

Una vez que se tiene esta red de flujo se le calcula el flujo maximo a travez de un algoritmo como ford-fulkerson, usando la variante de edmond-karp y esto nos maximizara las personas asignadas a cada tarea.

##### Ejemplo:

![5_3](img/5_3.png)

##### Complejidad:

La complejidad temporal sera la de el algoritmo de edmond-karp y es **O(V*E^2)**

La complejidad espacial sera **O(V+E)** ya que se necesita copiar el grafo original y agregar la superfuente y el supersumidero asi como las aristas correspontiendes. Y tambien se guardara el flujo en un diccionario de largo E (aristas del nuevo grafo).

##### Pseudocodigo:

```python
def create_flow_network(graph):
    flow_network = new Graph with nodes graph.all_nodes()
    create nodes source and sink
    for node in graph:
        if node is person:
            create_edge(source, person, infinity)
            for task in graph.adj(person):
                create_edge(person, task, 1)
        if node is task:
            create_edge(task, sink, task capacity)
    return flow_network

def max_flow(graph, s, t):
    flow = {}
    for v in graph:
        for w in graph.adj(v):
            flow[(v,w)] = 0
    
    residual = create_residual_graph(graph) # doing everything it means

    while (path = obtener_camino_menor_longitud(residual, s, t)) is not None:
        residual_flow = min_weight_of_all_edges(graph, path) # min(path_edges)
        for i in range(1, len(path)):
            if graph.has_edge(path[i-1], path[i]):
                flow[(path[i-1], path[i])] += residual_flow
                update_residual_graph(residual, path[i-1], path[i], residual_flow)
            else: # using antiparallel edge
                flow[(path[i], path[i-1])] -= residual_flow
                update_residual_graph(residual, path[i], path[i-1], residual_flow)
    
    return flow


def update_residual_graph(graph, u, v, flow):
    last_weight = graph.weight_of_edge(u, v)
    
    if last_weight == flow:
        graph.remove_edge(u, v)
    else:
        graph.update_edge(u, v, last_weight - flow)
    
    if not graph.has_edge(v, u): # antiparallel edge
        graph.add_edge(v, u, flow)
    else:
        graph.update_edge(v, u, graph.weight_of_edge(v, u) + flow)
```

### 4-

La red de transporte intergaláctico es una de las maravillas del nuevo imperio terráqueo. Cada tramo de rutas galácticas tiene una capacidad infinita de transporte entre ciertos planetas. No obstante, por burocracia - que es algo que no los enorgullece - existen puestos de control en cada planeta que reduce cuantos naves espaciales pueden pasar por día por ella. Por una catástrofe en el planeta X, la tierra debe enviar la mayor cantidad posible de naves de ayuda. Por un arreglo, durante un día los planetas solo procesaran en los puestos de control aquellas naves enviadas para esta misión.  Tenemos que determinar cuál es la cantidad máxima de naves que podemos enviar desde la tierra hasta el planeta X. Sugerencia: considerar a este un problema de flujo con capacidad en nodos y no en ejes


#### Solution

##### Algoritmo:

Se crea una red de flujo tal que la tierra es la fuente del flujo y el planeta X el sumidero y en el medio estaran los planetas intermedios entre los mismos. Teniendo en cuenta que en cada planeta hay un puesto de control donde se puede disminuir el flujo de naves, se calculara el flujo maximo a travez de la red utilizando un algoritmo ford-fulkerson buscando los caminos mas cortos primero, para eso se usara bfs (algoritmo edmond-karp). Una vez terminado sabremos hacia donde hay que mandar cada nave para maximizar las naves que llegan al planeta X.

##### Ejemplo:

![5_4](img/5_4.png)

##### Complejidad:

La complejidad temporal es **O(V*E^2)** por edmond-karp

La complejidad espacial es **O(V+E)** porque se le hace una copia al grafo para crear el residual, y el flujo se guarda en un diccionario de largo E.

##### Pseudocodigo:

```python
def max_ships(galactic_network, earth, planet_x):
    flow = initialize dict with all galactic_network.edges as keys and 0 as value

    residual = create_residual_graph(galactic_network)
    
    while path = get_shortest_path(residual):
        residual_flow = min of edges of the path # the weight

        for i in range(1, len(path)):
            if galactic_network.has_edge(path[i-1], path[i]):
                flow[(path[i-1], path[i])] += residual_flow
                update_residual_graph()
            else:
                flow[(path[i], path[i-1])] -= residual_flow
                update_residual_graph()
    return flow
```

### 5-

La compañía eléctrica de un país nos contrata para que le ayudemos a ver si su red de transporte desde su nueva generadora hidroeléctrica hasta su ciudad capital es robusta. Nos otorgan un plano con la red eléctrica completa: todas las subestaciones de distribución y red de cableados de alta tensión. Lo que quieren que le digamos es: cuantas secciones de su red se pueden interrumpir antes que la ciudad capital deje de recibir la producción de la generadora? (Sugerencia: investigue sobre el Teorema de Menger) Puede informar cual es el subconjunto de ejes cuya falla provoca este problema?

#### Solution

##### Algoritmo:

Dada la red de flujo de la compañia electrica, al buscar el corte minimo en la misma a travez de un algoritmo como ford-fulkerson o edmond-karp (buscando primero el flujo maximo, y luego nos fijamos hasta que nodos se puede llegar desde la fuente en el grafo residual, entonces las aristas que salgan de esos ultimos nodos pertenecen al conjunto del corte minimo) obtendriamos cuales son los cables que ocacionarian fallas y los cuales son los primeros que habria que mejorar, tambien si se quiere saber que cables pueden empeorar sin que afecte a la ciudad, estos son los que luego de calcular el flujo maximo todavia tienen capacidad restante.

##### Ejemplo:

![5_5](img/5_5.png)

##### Complejidad:

La complejidad es la misma de edmond-karp en caso de usar este y es **O(V*E^2)**

La complejidad espacial sera **O(V + E)** ya que se le hace una copia a la red de flujo para crear el grafo residual

##### Pseudocodigo:

```python
def min_cut(graph, s, t):
    init flow for every edge in graph at 0
    create residual graph

    while there is a path s,t in the residual graph: # with BFS
        residual flow = min weight of every edge in the path
        for edge in path:
            if edge in graph:
                flow of edge += residual flow
            else:
                flow of edge -= residual flow
            update residual graph acordingly
    
    # Once flow is calculated
    reachable nodes = get all reachable nodes from source in residual graph # with BFS
    furthest nodes = get furthest nodes from source in reachable nodes
    
    min cut = set()
    for node in furthest nodes:
        add all graph.adjacent(node) to the min cut
    
    return min cut
```


### 7-

La policía de la ciudad tiene “n” comisarías dispersas por la ciudad. Para un evento deportivo internacional deben asignar la custodia de “m” centros de actividades. Una comisaría y un centro de actividades pueden ser emparejados si y sólo si la distancia entre ellos no es mayor a un valor d. Contamos con la distancia entre todos los centros y las comisarías. Una comisaría sólo puede custodiar un centro. El centro puede ser custodiado por una comisaría. Determinar si es posible la asignación de tal forma que todos los centros estén custodiados. ¿Cómo modificaría la resolución del problema si en lugar de que cada centro de actividades i tenga que ser asignado a una sola comisaría, tenga que ser asignado a mi comisarías? ¿Cómo modificaría la resolución del problema si además hubiera una restricción entre comisarías que implicaría que una comisaría Ni y una Nj no pudieran ser asignadas juntas a un centro Mi? ¿Para qué casos dejaría de ser eficiente la resolución?

#### Solution

##### Algoritmo:

Primero se crea una red de flujo a partir del grafo donde a todas las estaciones de policia les llegara flujo de una superfuente, estas a su vez estaran conectadas con aristas a solamente los centros que les queda a distancia menor a d, y estos centros estaran conectados a un supersumidero. Las aristas tendran todas peso 1, ya que no queremos que una estacion de policias use mas de una arista porque eso significaria que esta cubriendo mas de un centro simultaneamente.

Una vez hecho esto se puede calcular el flujo maximo y si el flujo == m (y por consiguiente == n) entonces hay un perfect match en la red. Si se quiere saber que aristas se seleccionan basta con ver las aristas con capacidad al maximo.

##### Ejemplo:

![5_7](img/5_7.png)

##### Complejidad:

La complejidad es la de calcular el flujo maximo (ford-fulkerson o edmond-karp). Temporal **O(V*E^2)** y espacial **O(V + E)** por edmond-karp

##### Pseudocodigo:

*Igual a los flujos que ya hice arriba solo que en el return hay dar flujo_max == m*

### 8-

Una compañía minera nos pide que la ayudemos a analizar su nueva explotación. Ha realizado el estudio de suelos de diferentes vetas y porciones del subsuelo. Con estos datos se ha construido una regionalización del mismo. Cada región cuenta con un costo de procesamiento y una ganancia por extracción de metales preciosos. (En algunos casos el costo supera al beneficio). Al ser un procesamiento en profundidad ciertas regiones requieren previamente procesar otras para acceder a ellas. La compañía nos solicita que le ayudemos a maximizar su ganancia, determinando cuales son las regiones que tiene que trabajar. Tener en cuenta que el costo y ganancia de cada región es un valor entero. Para cada región sabemos cuales son aquellas regiones que le preceden. Resolver el problema planteado utilizando una aproximación mediante flujo de redes.

#### Solution

* Con ayuda de [Flor](https://github.com/fnpratto) jeje

##### Algoritmo:

Se toma la red dada y se usa edmond-karp para calcular flujo maximo, pero se hace un prepocesamiento a la red para que haya una fuente llendo a todos los nodos de grado de entrada 0 y un sumidero al que vayan todos los nodos de grado de salida 0. Luego a las aristas que tendrian como peso (costo, ganancia) las convierto en peso = ganancia - costo, y cuando una de negativa la pongo en 0, como tiene capacidad 0 no podra pasar flujo por ahi, es decir este algoritmo ignorara los caminos en que algun punto tengan costo > ganancia, inclusive aunq este camino de ser recorrido completo daria mas ganancia que costo, es poreso que este algoritmo da una aproximacion y no un resultado optimo. Una vez calculado el flujo maximo, todas las aristas que tengan flujo != 0 pasando por ellas del grafo original, estas son las que se debe atravesar.

##### Ejemplo:

![5_8](img/5_8.png)

##### Complejidad:

La complejidad temporal y espacial seran las mismas que edmond-karp **O(V*E^2)** y **O(V + E)** respectivamente

##### Pseudocodigo:

```python
def mine(graph, s, t):
    flow = edmondKarp(graph, s, t) # see previous exercises to see pseudocode for this part
    edges = set()
    for e in flow.keys():
        if flow[e] != 0:
            edges.add(e)
    return edges
```

### 10-

Una red de espías se encuentran diseminados por todo el país. Cada uno de ellos únicamente conoce a un número limitado de sus pares con los que pude tener contacto dejando un mensaje escrito en una ubicación conocida. Este conocimiento no es recíproco. En caso de una crisis la agencia puede enviar mensajes utilizando esta red desde su base principal a un determinado agente especial. Una cuestión importante es que una vez utilizado un espía para transmitir un mensaje durante el resto de la crisis no se vuelve a utilizar. La agencia desea conocer, dada su red y un agente de destino de sus mensajes. ¿Cuál es la mínima cantidad de espías que un rival podría neutralizar para reducir en un 30% la cantidad de mensajes máximos que puede enviar desde la base al agente? Utilizando redes de flujos dar una solución al problema.

#### Solution

##### Algoritmo:

Se usa la red dada para calcular el corte minimo a travez de un algoritmo de flujo como ford-fulkerson o edmond-karp, una vez obtenido el corte minimo vemos el final de cada una de estas aristas de corte en el grafo original, cada una tendra un espia que de ser asesinado reduciria el flujo. Tenemos que ir simulando asesinatos de espias contando el porcentaje en el que se reduce el flujo siendo 100% si asesinamos a todos los espias.

##### Ejemplo:

![5_10](img/5_10.png)

##### Complejidad:

La complejidad temporal y espacial sera la misma que edmond-karp

##### Pseudocodigo:

```python
def espias(grafo, s, t):
    flujo = {}
    for v in grafo:
        for w in grafo.adj(v):
            flujo[(v, w)] = 0
    
    residual = crear_grafo_residual(grafo)
    
    while camino = obtener_camino_menor_longitud(residual, s, t):
        flujo_residual = min(capacidad for arista in camino)

        for i in range(1, len(camino)):
            if grafo.tiene_arista((camino[i-1], camino[i])):
                flujo[(camino[i-1], camino[i])] += flujo_residual
                actualizar_residual(residual, camino[i-1], camino[i], flujo_residual)
            else:
                flujo[(camino[i], camino[i-1])] -= flujo_residual
                actualizar_residual(residual, camino[i], camino[i-1], flujo_residual)
    
    furthest_reachable_spies = get_furthest_reachable_nodes(residual, s)
    min_cut_edges = set()
    for v in furthest_reachable_spies:
        for w in grafo.adj(v):
            min_cut_edges.add((v, w))
    
    espias_asesinados = 0
    while floor(espias_asesinados / len(min_cut_edges))*100 < 30:
        espias_asesinados += 1
    
    return espias_asesinados # numero de espias a asesinar para disminuir en 30% el flujo
```

### 11-

En un juego multijugador cooperativo de "p" participantes se muestra una grilla de n*n celdas. Inicialmente en una posición al azar se colocan los avatares de los jugadores e igual cantidad de cuevas. Cada "gusano" se desplaza 1 celda por turno ocupando una circundante que esté vacía (como máximo tiene 4: arriba, abajo, izquierda y derecha). A medida que se desplaza crece y por lo tanto sigue ocupando por las que pasó anteriormente. El objetivo del juego es lograr que los “p” jugadores lleven a sus gusanos a las cuevas (es indiferente a cual pero cada uno tiene que estar en una cueva diferente). Nos solicitan que realicemos el pseudocódigo que verifique en un turno determinado si aún es posible que los jugadores ganen. Es decir que todos los gusanos puedan llegar a la cueva.

#### Solution

##### Algoritmo:

El algoritmo consiste en primeramente crear un grafo donde cada cabeza de serpiente este recibiendo flujo de una fuente, el peso de esa arista sera 1. Tambien cada cabeza estara conectado con aristas de peso 1 a sus posibles jugadas (wasd) luego ese nodo estara conectado a los posibles lugares a los que se puede ir desde ahi y asi sucesivamente. se calcula el flujo y se ve si es igual al numero de jugadores en ese caso es True sino es False.

##### Ejemplo:

![5_11](img/5_11.png)

##### Complejidad:

La complejidad temporal sera **O(V*E^2)** por calcular el flujo con edmond-karp

La complejidad espacial es **O(V+E)**

##### Pseudocodigo:

```python
def puede_ganar(grafo, s, t):
    return flujo(grafo, s, t) == cant_jugadores
```

### 13-

Una empresa de autobuses se conformó luego de la fusión de varias compañías menores. Actualmente tienen diferentes rutas que cubrir. Cada una con horario de inicio en una ciudad y finalización en otra. Existe la posibilidad de cubrir con un mismo micro diferentes rutas. Siempre la ruta comienza desde donde parte el micro, pero también puede pasar que el micro tenga tiempo suficiente para trasladarse hasta otro punto y cubrir otra ruta. Cuentan con una flota activa de N micros. Necesitan saber si les es posible cubrir con ella los requerimientos y si pueden contar con micros de backup ante la necesidad de controles programados de algunos de los móviles. Ayudar a resolver el problema mediante el uso de redes de flujo.

#### Solution

##### Algoritmo:

Se crea una red de flujo donde todos los origenes estaran conectados a una fuente con aristas de capacidad infinita, y todos los destinos estaran conectados al sumidero con aristas de capacidad 1. A su vez entre origenes y destinos se conectaran con aristas tambien de capacidad 1, pero se agregaran aristas desde los destinos hacia los origenes de otros viajes siempre y cuando se pueda usar el mismo micro para ese viaje, es decir que de con el tiempo, estas nuevas aristas tendran como capacidad minima 0 (no se usa ese micro para este nuevo viaje) y como capacidad maxima 1 (si se usa).

Al maximizar el flujo por medio de un algoritmo como ford-fulkerson o edmond-karp maximizaremos los micros en uso, pero como la fuente da flujo infinito este podria decirnos que usemos mas de los que tenemos, poreso la fuente se conecta a otra fuente que le da flujo N. Asi garantizamos que no se pase de los micros existentes.

##### Ejemplo:

![5_13](img/5_13.png)

##### Complejidad:

La complejidad temporal y espacial sera la misma que edmond-karp

##### Peudocodigo:

**Calcular flujo maximo y si este es igual a la cantidad de pares origen-destino entonces se pudo hacer todos los viajes**


#### Skipped 6, 9, 12, 14, 15